In [1]:
from sahi import AutoDetectionModel
from sahi.predict import get_sliced_prediction
import cv2
import os
from ultralytics import YOLO
import time

In [2]:
YOLO_MODELS = [
    'yolov5m6u.pt',  # Ultra-real-time optimized
    'yolov8m.pt',    # Balanced accuracy/speed
    'yolov8x.pt',    # High accuracy
    'yolov9e.pt'     # Best Performer (Target Model)
]

In [3]:
models = ["/home/desly/DefectClassification/models/yolov5m6u.pt", "/home/desly/DefectClassification/models/yolov8m.pt","/home/desly/DefectClassification/models/yolov8x.pt", "/home/desly/DefectClassification/models/yolov9e.pt"]
image = r"/home/desly/DefectClassification/pictures/animals.jpg"
img = cv2.imread(image)
height, width, channels = img.shape

print(f"Width: {width}, Height: {height}, Channels: {channels}")

resized_img = cv2.resize(img, (500, 300))

# Verify the new size
h, w, c = resized_img.shape
print(f"New Dimensions - Width: {w}, Height: {h}")

Width: 612, Height: 366, Channels: 3
New Dimensions - Width: 500, Height: 300


In [8]:
# List to store timing results for comparison later
timing_report = []

for model in models:
    model_name = model.split("/")[-1].removesuffix(".pt")
    print(f"--- Processing: {model_name} ---")

    detection_model = AutoDetectionModel.from_pretrained(
        model_type='ultralytics',
        model_path=model,
        confidence_threshold=0.66,  #results with 66 percent surety
        device='cpu' # "cuda" for gpu
    )


    start_time = time.perf_counter()

    results = get_sliced_prediction(
        resized_img,
        detection_model,
        slice_height=100,
        slice_width=100,
        overlap_height_ratio=0.1,
        overlap_width_ratio=0.1
    )

    end_time = time.perf_counter()
    elapsed_time = end_time - start_time

    print(f"Inference Time: {elapsed_time:.4f} seconds")
    
    timing_report.append({"model": model_name, "time": elapsed_time})

    results.export_visuals(
        export_dir="demo_results/",
        file_name=f"Sahi_prediction_{model_name}"
    )


    for prediction in results.object_prediction_list:
        class_name = prediction.category.name
        score = prediction.score.value
        print(f" - {class_name}: {score:.2%}")

# --- Summary ---
print("\n--- Performance Summary ---")
for entry in timing_report:
    print(f"{entry['model']}: {entry['time']:.4f}s")

--- Processing: yolov5m6u ---
Performing prediction on 24 slices.
Inference Time: 121.9171 seconds
 - person: 70.19%
 - bird: 79.94%
 - cat: 87.71%
 - dog: 82.57%
--- Processing: yolov8m ---
Performing prediction on 24 slices.
Inference Time: 30.5298 seconds
 - bird: 93.12%
 - cat: 85.58%
 - dog: 81.36%
--- Processing: yolov8x ---
Performing prediction on 24 slices.
Inference Time: 86.6101 seconds
 - bird: 93.41%
 - cat: 87.18%
 - dog: 80.66%
--- Processing: yolov9e ---
Performing prediction on 24 slices.
Inference Time: 118.1066 seconds
 - bird: 92.34%
 - bird: 85.05%
 - cat: 87.64%
 - dog: 87.09%

--- Performance Summary ---
yolov5m6u: 121.9171s
yolov8m: 30.5298s
yolov8x: 86.6101s
yolov9e: 118.1066s


In [5]:
os.makedirs("demo_results", exist_ok=True)

timing_report = []

for model_path in models:
    model_name = model_path.split("/")[-1].removesuffix(".pt")
    print(f"--- Processing: {model_name} ---")

  
    model = YOLO(model_path)

    start_time = time.perf_counter()

    
    results = model.predict(
        source=resized_img, 
        conf=0.66,      # 66% confidence threshold
        device='cpu',   # or '0' for GPU
        verbose=True
    )

    end_time = time.perf_counter()
    elapsed_time = end_time - start_time

    print(f"Inference Time: {elapsed_time:.4f} seconds")
    timing_report.append({"model": model_name, "time": elapsed_time})

   
    result = results[0]

    plotted_img = result.plot()
    save_path = os.path.join("demo_results", f"Direct_prediction_{model_name}.jpg")
    cv2.imwrite(save_path, plotted_img)

    if len(result.boxes) > 0:
        for box in result.boxes:
            # box.cls is a tensor, we convert to int
            class_id = int(box.cls[0])
            class_name = model.names[class_id]
            # box.conf is a tensor, we convert to float
            score = float(box.conf[0])
            print(f" - {class_name}: {score:.2%}")
        print("")
    else:
        print(" - No detections found.")

# --- Summary ---
print("\n--- Performance Summary ---")
for entry in timing_report:
    print(f"{entry['model']}: {entry['time']:.4f}s")

--- Processing: yolov5m6u ---

0: 768x1280 1 bird, 2 cats, 2 dogs, 1 bowl, 3829.5ms
Speed: 19.4ms preprocess, 3829.5ms inference, 4.6ms postprocess per image at shape (1, 3, 768, 1280)
Inference Time: 4.3746 seconds
 - dog: 91.05%
 - dog: 90.68%
 - cat: 88.91%
 - bird: 87.49%
 - cat: 85.52%
 - bowl: 74.48%

--- Processing: yolov8m ---

0: 384x640 2 birds, 3 cats, 4 dogs, 1 bowl, 945.7ms
Speed: 4.1ms preprocess, 945.7ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)
Inference Time: 1.2342 seconds
 - dog: 95.79%
 - cat: 90.58%
 - dog: 90.31%
 - bird: 85.66%
 - cat: 77.86%
 - cat: 77.69%
 - bird: 73.49%
 - dog: 73.35%
 - bowl: 71.56%
 - dog: 70.41%

--- Processing: yolov8x ---

0: 384x640 2 birds, 3 cats, 2 dogs, 2186.2ms
Speed: 10.9ms preprocess, 2186.2ms inference, 10.9ms postprocess per image at shape (1, 3, 384, 640)
Inference Time: 3.1020 seconds
 - dog: 94.08%
 - bird: 91.79%
 - cat: 90.64%
 - cat: 85.62%
 - cat: 84.96%
 - bird: 81.80%
 - dog: 68.41%

--- Processin